In [3]:
!pip install selenium
!pip install webdriver_manager

  Obtaining dependency information for docker from https://files.pythonhosted.org/packages/db/be/3032490fa33b36ddc8c4b1da3252c6f974e7133f1a50de00c6b85cca203a/docker-6.1.3-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 8.1 MB/s eta 0:00:00


##  Convert the rise test in makefile to ipynb

In [7]:
# Cell 1: Imports and preparations
import subprocess
import selenium

# Function to run shell commands and capture output
def run_shell_command(cmd):
    try:
        result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True, shell=True)
        return result.stdout.decode('utf-8').strip()
    except subprocess.CalledProcessError as e:
        return e.stderr.decode('utf-8').strip()

# Cell 2: Get Docker container ID
container_id_command = "docker ps -q | head -n 1"
container_id = run_shell_command(container_id_command)
container_id  # This will display the container ID

# Cell 3: Get JupyterLab token
token_command = f"docker exec {container_id} jupyter lab list 2>&1 | grep -o 'token=[^ ]*' | cut -d= -f2"
token = run_shell_command(token_command)
token  # This will display the token

# Cell 4: Run the rise_test.py script
# You need to have the rise_test.py file in the current directory of the Jupyter notebook.
rise_test_command = f"python3 rise_test.py {token}"
result = run_shell_command(rise_test_command)
result  # This will display the output of the script

''

##  Get the container ID

In [6]:
import subprocess

# Function to run shell commands
def run_command(command):
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    if process.returncode != 0:
        print(f"Error: {stderr.decode('utf-8')}")
        return None
    return stdout.decode('utf-8').strip()

# Get the CONT_ID equivalent in Python
cont_id_command = "docker ps -q | head -n 1"
cont_id = run_command(cont_id_command)
print(f"CONT_ID: {cont_id}")

# Get the TOKEN equivalent in Python
if cont_id:
    token_command = f"docker exec {cont_id} jupyter lab list 2>&1 | grep -o 'token=[^ ]*' | cut -d= -f2"
    token = run_command(token_command)
    print(f"TOKEN: {token}")
    
    # Run the rise_test.py script with the token
    if token:
        rise_test_command = f"python3 rise_test.py {token}"
        print(run_command(rise_test_command))


CONT_ID: 


## Get the token Run the rise_test.py 

In [8]:
# First cell: Set up imports and variables
import subprocess
import os

def get_container_id():
    return subprocess.check_output('docker ps -q | head -n 1', shell=True).decode().strip()

def get_token(container_id):
    cmd = f'docker exec {container_id} jupyter lab list 2>&1'
    output = subprocess.check_output(cmd, shell=True).decode()
    token = [line for line in output.split('\n') if "token=" in line][0].split('=')[1]
    return token

CONT_ID = get_container_id()

# Next cell: Get TOKEN and run rise_test.py with it
if CONT_ID:
    TOKEN = get_token(CONT_ID)
    if TOKEN:
        # Here you replace 'path/to/rise_test.py' with the actual path to the Python script
        result = subprocess.run(['python3', 'path/to/rise_test.py', TOKEN], capture_output=True, text=True)
        print(result.stdout)
    else:
        print("Token not found. Check if Jupyter Lab is running in the container.")
else:
    print("No running Docker container found.")


No running Docker container found.


/bin/sh: 1: docker: not found


## Check Docker

In [9]:
import subprocess

def check_docker():
    try:
        subprocess.check_output(['docker', '--version'])
        return True
    except (subprocess.CalledProcessError, FileNotFoundError):
        return False

def get_container_id():
    try:
        return subprocess.check_output('docker ps -q | head -n 1', shell=True).decode().strip()
    except subprocess.CalledProcessError:
        print("Docker command failed. Ensure Docker is running and you have the right permissions.")
        return None

def get_token(container_id):
    try:
        cmd = f'docker exec {container_id} jupyter lab list 2>&1'
        output = subprocess.check_output(cmd, shell=True).decode()
        token = [line for line in output.split('\n') if "token=" in line][0].split('=')[1]
        return token
    except subprocess.CalledProcessError:
        print("Failed to get token. Ensure Jupyter Lab is running in the container.")
        return None

if check_docker():
    CONT_ID = get_container_id()
    if CONT_ID:
        TOKEN = get_token(CONT_ID)
        if TOKEN:
            # Here you replace 'path/to/rise_test.py' with the actual path to the Python script
            result = subprocess.run(['python3', 'path/to/rise_test.py', TOKEN], capture_output=True, text=True)
            print(result.stdout)
        else:
            print("Token not found. Check if Jupyter Lab is running in the container.")
    else:
        print("No running Docker container found.")
else:
    print("/bin/sh: 1: docker: not found. Ensure Docker is installed and available in PATH.")


/bin/sh: 1: docker: not found. Ensure Docker is installed and available in PATH.
